In [212]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [213]:
data = pd.read_csv("iris.csv")
header = ["sepal.length", "sepal.width", "petal.length", "petal.width"]

# Approaching the problem
Decision Tree is almost identical to when we want to decide a thing. For example, if we want to go swimming, we might consider if it's raining or not, or is the pool indoor or outdoor, etc.. However, computer obviously can't think about the question to ask in order to decide the outcome.

That's why we need a criterion to decide what is the best question to ask. GINI impurity score will be implemented here. This is called choosing the best split.

Like most tree building algorithm, we can build the tree recursively and for the base case, there are many complicated arguments involved, such as max depth of a tree, the min number of sample in a leaf, etc.. But those won't be considered and we will build the tree until no more leaf can be created.

First, let's implement Gini. Gini impurity can be calculated by summing pairwise products of these probabilities for each class label, according to Wiki:

$$ 1 - \sum^{N}_{i=1} p_i^2 $$

where N is the class count in the dataset and $p_i$ is the probability of choosing an item with label $i$

In [214]:
def label_count(data):
    label_count = {}

    if isinstance(data, pd.DataFrame):
        for _, row in data.iterrows():
            label = row[-1]
            if label not in label_count:
                label_count[label] = 0
            label_count[label] += 1
    else:
        for row in data:
            label = row[-1]
            if label not in label_count:
                label_count[label] = 0
            label_count[label] += 1

    return label_count

In [215]:
label_count(data)

{'Setosa': 50, 'Versicolor': 50, 'Virginica': 50}

In [216]:
def gini(data):
    label_count_ = label_count(data)
    gini = 1
    for label in label_count_:
        prob = label_count_[label] / len(data)
        gini -= prob**2
    return gini


def info_gain(left, right, current_purity):
    w = float(len(left)) / (len(left) + len(right))
    return current_purity - w * gini(left) - (1 - w) * gini(right)

In [217]:
no_mixing = [["True"], ["True"]]
# this will return 0
gini(no_mixing)

0.0

In [218]:
mixing = [["True"], ["False"]]
# this will return 0.5
gini(mixing)

0.5

# Helper function
We need a function that can find the best question. More specifically, we can find the Gini score for each predictor and select the one with lowest impurity.

In [219]:
def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)


class Question:
    def __init__(self, predictor, target):
        self.predictor = predictor
        self.target = target

    def match(self, row):
        val = row[self.predictor]
        if is_numeric(val):
            return val >= self.target
        else:
            return val == self.target

    def __repr__(self):
        condition = "=="
        if is_numeric(self.target):
            condition = ">="
        return f"Is {header[self.predictor]} {condition} {str(self.target)}?"

In [220]:
# This question will ask if sepal.length (index 1) >= 5
q = Question(1, 5)
q

Is sepal.width >= 5?

In [221]:
example = data.iloc[3]
print(example)
q.match(example)
# this will return False as sepal.length is 4.6

sepal.length       4.6
sepal.width        3.1
petal.length       1.5
petal.width        0.2
variety         Setosa
Name: 3, dtype: object


False

Now we want to define the function that can partition the result of a question into 2 category True and False. Also, we want to have a function that can choose the best split for our decision nodes

In [222]:
def partition(data, question):
    true_rows, false_rows = [], []
    for _, row in data.iterrows():
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [223]:
true_rows, false_rows = partition(data, Question(1, 4))
true_rows

[sepal.length       5.8
 sepal.width        4.0
 petal.length       1.2
 petal.width        0.2
 variety         Setosa
 Name: 14, dtype: object,
 sepal.length       5.7
 sepal.width        4.4
 petal.length       1.5
 petal.width        0.4
 variety         Setosa
 Name: 15, dtype: object,
 sepal.length       5.2
 sepal.width        4.1
 petal.length       1.5
 petal.width        0.1
 variety         Setosa
 Name: 32, dtype: object,
 sepal.length       5.5
 sepal.width        4.2
 petal.length       1.4
 petal.width        0.2
 variety         Setosa
 Name: 33, dtype: object]

In [224]:
def best_split(data):
    best_gain = 0
    best_question = None
    current_impurity = gini(data)
    n_features = len(data.columns) - 1

    for col in range(n_features):
        values = set(data.iloc[:, col])
        for val in values:
            question = Question(col, val)
            true_rows, false_rows = partition(data, question)
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue
            gain = info_gain(true_rows, false_rows, current_impurity)
            if gain >= best_gain:
                best_gain, best_question = gain, question
    return best_gain, best_question

# Classes for the tree

In [229]:
class Leaf:
    def __init__(self, data):
        self.label_count_ = label_count(data)

    def print(self):
        total = sum(self.label_count_.values()) * 1.0
        for label in self.label_count_:
            print(label, self.label_count_[
                  label] / total * 100, "%", end=" | ")
        print()


class DecisionNode:
    def __init__(self, question, true_branch, false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

# Building a tree
1. Iterate through the dataset and select the best split (having the lowest purity) to be the root.
2. If the information gain is not 0, partition into 2 child nodes. Else that is a leaf node.
3. Repeat the process above

In [226]:
def build_tree(data):
    best_gain, best_question = best_split(data)
    if best_gain == 0:
        return Leaf(data)
    true_rows, false_rows = partition(data, best_question)
    true_branch = build_tree(pd.DataFrame(true_rows))
    false_branch = build_tree(pd.DataFrame(false_rows))
    return DecisionNode(best_question, true_branch, false_branch)

In [227]:
def print_tree(node, spacing=""):
    if isinstance(node, Leaf):
        print(spacing, end="")
        node.print()
        return
    print(spacing + str(node.question))
    print(spacing + "--> True:")
    print_tree(node.true_branch, spacing + "  ")
    print(spacing + "--> False:")
    print_tree(node.false_branch, spacing + "  ")

In [230]:
print_tree(build_tree(data))

Is petal.width >= 1.0?
--> True:
  Is petal.width >= 1.8?
  --> True:
    Is petal.length >= 4.9?
    --> True:
      Virginica 100.0 % | 
    --> False:
      Is sepal.width >= 3.2?
      --> True:
        Versicolor 100.0 % | 
      --> False:
        Virginica 100.0 % | 
  --> False:
    Is petal.length >= 5.0?
    --> True:
      Is petal.width >= 1.6?
      --> True:
        Is petal.length >= 5.8?
        --> True:
          Virginica 100.0 % | 
        --> False:
          Versicolor 100.0 % | 
      --> False:
        Virginica 100.0 % | 
    --> False:
      Is petal.width >= 1.7?
      --> True:
        Virginica 100.0 % | 
      --> False:
        Versicolor 100.0 % | 
--> False:
  Setosa 100.0 % | 


# Reference
Let’s Write a Decision Tree Classifier from Scratch - Machine Learning Recipes #8. Retrieved from https://www.youtube.com/watch?v=LDRbO9a6XPU

Decision Tree learning Wikipedia. Retrieved from https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity